In [35]:
import dgl
import torch
import torch.nn as nn
from dgl.nn import GINEConv,GATConv
import numpy as np

In [66]:
g=dgl.graph(([0,1,2,1],
             [1,2,1,0]))
in_feat=2
out_feat=2
n_feat=torch.randn(g.num_nodes(),in_feat)
e_feat=torch.randn(g.num_edges(),in_feat)
print(n_feat.shape)
print(e_feat.shape)

torch.Size([3, 2])
torch.Size([4])


In [32]:
print(n_feat)
print(e_feat)
conv=GINEConv(nn.Linear(in_feat,out_feat))
res=conv(g,n_feat,e_feat)
print(res)

tensor([[ 0.5888, -0.8675,  0.4436],
        [ 2.4353, -2.4259, -0.3092],
        [ 0.2220, -2.3916,  1.8348]])
tensor([[-0.4855,  0.0899, -1.2415],
        [ 1.0146, -0.0956,  0.1709],
        [ 0.9142,  0.9067, -0.5644],
        [-0.8427,  1.1041,  1.7460]])
tensor([[ 1.0899, -1.3193],
        [ 0.1359, -1.3522],
        [ 0.6151, -1.8083]], grad_fn=<AddmmBackward0>)


In [79]:
a=torch.randn((10,10,3))
b=torch.randn((10,10,3))
c=a.detach()

In [81]:
cos=nn.CosineSimilarity(0)
output=torch.cosine_similarity(a.flatten(),b.flatten(),dim=0)
print(output.item())

-0.09277930855751038


In [ ]:
g=dgl.graph(([0,1,2,1],
             [1,2,1,0]))
in_feat=2
out_feat=2
n_feat=torch.randn(g.num_nodes(),in_feat)
e_feat=torch.randn(g.num_edges(),in_feat)

In [67]:
gat1=GATConv(in_feat,10,2)
pred=gat1(g,n_feat,e_feat,get_attention=True)